In [20]:
import asyncio
# import psycopg
from sqlalchemy.ext.asyncio import create_async_engine, async_sessionmaker, AsyncSession
from sqlalchemy.orm import Session, sessionmaker
from sqlalchemy import text, create_engine, select, MetaData
from config import settings

In [21]:
# Настройка синхронного двигателя
sync_engine = create_engine(
    url=settings.DATABASE_URL_psycopg,
    echo=False,
    pool_size=5,
    max_overflow=10,
)

# Настройка асинхронного двигателя
async_engine = create_async_engine(
    url=settings.DATABASE_URL_asyncpg,
    echo=False,
    pool_size=5,
    max_overflow=10,
)

# Создание фабрик сессий
sync_session = sessionmaker(sync_engine)
async_session = async_sessionmaker(async_engine, expire_on_commit=False)

metadata = MetaData()

In [22]:
async def get_async():
    # Создание асинхронной сессии
    async with async_session() as session:
        async with session.begin():  # Начало транзакции
            # Выполнение запроса
            result = await session.execute(text("SELECT VERSION()"))  
            version = result.scalar_one()  # Получение одного значения
            print(f"Database version: {version}")

            # Повторный запрос для получения строк, если требуется
            result = await session.execute(text("SELECT VERSION()"))  
            first_row = result.first()
            print(f'{first_row=}')  # Первая строка результата (если есть)

            # Итерация по результату
            result = await session.execute(text("SELECT VERSION()"))
            for row in result:
                print(f"Database version: {row[0]}")


In [23]:
# Запуск асинхронной функции
# asyncio.run(get_async())

await get_async()

Database version: PostgreSQL 17.2 (Debian 17.2-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit
first_row=('PostgreSQL 17.2 (Debian 17.2-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit',)
Database version: PostgreSQL 17.2 (Debian 17.2-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit


In [24]:
data = {'lesson_id': 1,
 'students_id': [4, 5, 6],
 'training_check': [{'repetitions': 5, 'training_id': 1},
                    {'repetitions': 8, 'training_id': 2}]}

print(data.get("lesson_id"))
student_ids=data.get("students_id")
print(student_ids)

data2 = {'lesson_id': 1,
         'student_ids': data.get("students_id"),
         'training_check': data.get("training_check")}

print(data2)

1
[4, 5, 6]
{'lesson_id': 1, 'student_ids': [4, 5, 6], 'training_check': [{'repetitions': 5, 'training_id': 1}, {'repetitions': 8, 'training_id': 2}]}


In [44]:
from pprint import pprint

async def get_async():
    # Создание асинхронной сессии
    async with async_session() as session:
        async with session.begin():  # Начало транзакции
            # Выполнение запроса
            result = await session.execute(text('SELECT * FROM "Users" t where id = 4;'))  
            
            # Получение всех строк
            rows = result.fetchall()
            
            # Преобразование каждой строки в словарь
            rows_as_dict = [dict(row._mapping) for row in rows]

            pprint(rows_as_dict)
            

            

# Запуск асинхронной функции
# asyncio.run(get_async())

await get_async()

[{'avatar': None,
  'birthday': datetime.datetime(2020, 6, 10, 15, 41, 16, 490000),
  'code': None,
  'contact': 'string',
  'date_add': datetime.datetime(2024, 12, 31, 16, 23, 6, 416639),
  'date_update': datetime.datetime(2024, 12, 31, 16, 23, 6, 416643),
  'deleted': False,
  'email': 'student50@example.com',
  'extended_id': None,
  'firstname': 'student50',
  'hashed_password': b'$2b$12$VwFxBanEqA7aM.opPtBceOEkHlSwODjrhkSFABNUqO36vG6yY'
                     b'fKQm',
  'id': 4,
  'is_active': True,
  'is_man': True,
  'is_superuser': False,
  'is_verified': True,
  'lastname': 'studentov50',
  'photo': None,
  'rank_id': None,
  'role': 'student',
  'surname': 'studentovich50'}]
